In [6]:
import pandas as pd
from sklearn import preprocessing
from sklearn import metrics
import xgboost as xgb
df = pd.read_csv("input/adult.csv")
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [11]:
def run(fold):
    df = pd.read_csv("input/adult.csv")
    num_cols =[
         "fnlwgt",
         "age",
         "capital.gain",
         "capital.loss",
         "hours.per.week"
     ]
    df = df.drop(num_cols, axis=1)
    
    target_mapping = {
        "<=50K": 0,
        ">50": 1
    }
    df.loc[:,"income"] = df.income.map(target_mapping)
    features = [
        f for f in df.columns if f not in ("kfold","income")
    ]
    
    for col in features:
        df.loc[:,col] = df[col].astype(str).fillna("NONE")
    for col in features:
        lbl = preprocessing.LabelEncoder()
        lbl.fit(df[col])
        df.loc[:,col] = lbl.transform(df[col])
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_train = df[df.kfold == fold].reset_index(drop=True)
    x_train = df_train[features].values
    x_valid = df_valid[features].values
    
    model = xgb.XGBClassifier(
        n_jobs = -1
    )
    model.fit(x_train, df_train.income.values)
    
    valid_preds = model.predict_proba(x_valid)[:,1]
    
    auc = metrics.roc_auc_score(df_valid.income.values, valid_preds)
    print(f"Folds = {fold}, AUC = {auc}")
if __name__ == "__main__":
    for fold_ in range(5):
        run(fold_)

AttributeError: 'DataFrame' object has no attribute 'kfold'